In [1]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn

In [2]:
from huggingface_hub import login

#Enter your token here
HF_TOKEN = ''

login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
llama_path = 'meta-llama/Llama-2-13b-hf'
llama_model = AutoModelForCausalLM.from_pretrained(llama_path, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(llama_path, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
INTERVAL = 2
MERGE_LAYERS = 6
HIGHEST_LAY = 40
LOWEST_LAY = 1
THRESHOLD = 0.75

In [6]:
from copy import deepcopy
def merge_layers_return_model(model, merge_base_lay, merge_layer_num):

    merge_layer_num = min(merge_layer_num, len(model.model.layers) - merge_base_lay - 1)

    model_copy = deepcopy(model)
    for diff_lay in range(merge_base_lay+1, merge_base_lay+1+merge_layer_num):
        # gate_proj
        model_copy.model.layers[merge_base_lay].mlp.gate_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.gate_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.gate_proj.weight.data
        )
        # down_proj
        model_copy.model.layers[merge_base_lay].mlp.down_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.down_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.down_proj.weight.data
        )
        # up_proj
        model_copy.model.layers[merge_base_lay].mlp.up_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.up_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.up_proj.weight.data
        )


        # q_proj
        model_copy.model.layers[merge_base_lay].self_attn.q_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.q_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.q_proj.weight.data
        )

        # k_proj
        model_copy.model.layers[merge_base_lay].self_attn.k_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.k_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.k_proj.weight.data
        )

        # v_proj
        model_copy.model.layers[merge_base_lay].self_attn.v_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.v_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.v_proj.weight.data
        )

        # o_proj
        model_copy.model.layers[merge_base_lay].self_attn.o_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.o_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.o_proj.weight.data
        )

    for diff_lay in range(merge_base_lay+merge_layer_num, merge_base_lay, -1):

        del(model_copy.model.layers[diff_lay])
    return model_copy

In [7]:
import copy
llama_copy_to_compress = copy.deepcopy(llama_model)

In [8]:
import numpy as np
def cal_last_hidden_sim(model1, model2, tokenizer, sents):
    sim_ls = []
    for s in sents:
        encoded_inputs = tokenizer(s, return_tensors='pt')
        with torch.no_grad():
            # Disable cache during forward pass
            outputs1 = model1(**encoded_inputs, output_hidden_states=True, use_cache=False)
            hidden_states1 = outputs1.hidden_states[-1]
        with torch.no_grad():
            outputs2 = model2(**encoded_inputs, output_hidden_states=True, use_cache=False)
            hidden_states2 = outputs2.hidden_states[-1]

        similarity = torch.cosine_similarity(
            hidden_states1.squeeze(0).flatten().unsqueeze(0),
            hidden_states2.squeeze(0).flatten().unsqueeze(0)
        )
        sim_ls.append(similarity)

    sim_ls = [i.item() for i in sim_ls]
    print("Similarities:", sim_ls, "Mean similarity:", np.mean(sim_ls))
    return np.mean(sim_ls)



In [9]:
lay = HIGHEST_LAY - MERGE_LAYERS
last_merge_flag = False

sents = []
en_wiki_selected = ['Mouron () is a commune in the Arde',
 'The 81st Mechanised Brigade () is a mechanised brigade of the Romanian Land Force',
 'There are 18 National Natural Landmarks in the U.S. state of Washington, out of nearly',
 'Torreorgaz is a municipality in the',
 'Copa Libertadores 1973 was won by defending champions Independiente of A']

# zh_wiki_selected = ['月桃   \xa0\xa0月桃月桃属草本，单叶，互生，具',
#  '法国立贝尔洁白牙贴  目录产品成份：产品功效：用法用量：注意事项：产品禁忌：不良反应：规\xa0 \xa0 格：医疗器械注册号：产品执行标准：生产许可证号：授权监制：生产企业：',
#  'TIMKEN 641/632-B轴承  目录TIMK',
#  '天然碳化物质微结构研究  目录图书信息内容简介  图书信息作\u3000\u3000者： 冯有利 著 \n丛 书 名：\xa0\xa0出 版 社： 地质出版社 ISBN：9787116059771 出版时间',
#  'V字领衣服  目录基本信息']

sents.extend(en_wiki_selected)
# sents.extend(zh_wiki_selected)


while lay >= LOWEST_LAY:
    print(lay)
    print('current model layer', len(llama_copy_to_compress.model.layers))
    tmp_merged_model = merge_layers_return_model(llama_copy_to_compress, lay, MERGE_LAYERS-1)
    sim_value = cal_last_hidden_sim(llama_model, tmp_merged_model, tokenizer, sents)
    if sim_value > THRESHOLD:
        llama_copy_to_compress = tmp_merged_model
        lay -= INTERVAL
        if lay >= len(llama_copy_to_compress.model.layers):
            lay = len(llama_copy_to_compress.model.layers) - 1 - MERGE_LAYERS
    else:
        lay -= 1

34
current model layer 40
Similarities: [0.7511426210403442, 0.7406703233718872, 0.6075266599655151, 0.7468674182891846, 0.6956140995025635] Mean similarity: 0.7083642244338989
33
current model layer 40
Similarities: [0.7967424392700195, 0.784761905670166, 0.7845500111579895, 0.7877334356307983, 0.7450882196426392] Mean similarity: 0.7797752022743225
31
current model layer 35
Similarities: [0.664319634437561, 0.6455082893371582, 0.4992692768573761, 0.6480304002761841, 0.5746638774871826] Mean similarity: 0.6063582956790924
30
current model layer 35
Similarities: [0.6009887456893921, 0.600375771522522, 0.4468846917152405, 0.587017834186554, 0.5374423265457153] Mean similarity: 0.5545418739318848
29
current model layer 35
Similarities: [0.583367109298706, 0.5834397077560425, 0.43594425916671753, 0.5759263038635254, 0.5117914080619812] Mean similarity: 0.5380937576293945
28
current model layer 35
Similarities: [0.6629320383071899, 0.6567419767379761, 0.6100192070007324, 0.6550266742706299

In [10]:
llama_copy_to_compress.config.num_hidden_layers = len(llama_copy_to_compress.model.layers)
llama_copy_to_compress

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-34): 35 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
      )
    )
    (norm):

In [11]:
llama_copy_to_compress.config.num_hidden_layers

35

In [12]:
llama_copy_to_compress.save_pretrained("/content/pruned_model")
tokenizer.save_pretrained("/content/pruned_model")

('/content/pruned_model/tokenizer_config.json',
 '/content/pruned_model/special_tokens_map.json',
 '/content/pruned_model/tokenizer.model',
 '/content/pruned_model/added_tokens.json',
 '/content/pruned_model/tokenizer.json')

In [13]:
with open("/content/pruned_model/README.md", "w") as f:
    f.write("# Pruned LLaMA Model\n\nThis is a pruned version of the LLaMA2-13b model.")

In [14]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="namannn/llama2-13b-laco", exist_ok=True)
api.upload_folder(
    folder_path="/content/pruned_model",
    repo_id="namannn/llama2-13b-laco",
    repo_type="model",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:3757: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


model-00009-of-00010.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00008-of-00010.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00007-of-00010.safetensors:   0%|          | 0.00/4.79G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/namannn/llama2-13b-laco/commit/21c0f1f61002fbac4a4b2d9f84ac38e7a7aa4383', commit_message='Upload folder using huggingface_hub', commit_description='', oid='21c0f1f61002fbac4a4b2d9f84ac38e7a7aa4383', pr_url=None, pr_revision=None, pr_num=None)